In [ ]:
%matplotlib inline

import os
import h5py
import pandas as pd
import numpy as np
import pickle as pkl
import time

import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils import to_categorical
from statsmodels.stats.proportion import proportion_confint
from sklearn.metrics import confusion_matrix
from sklearn import metrics

import tensorflow as tf
from tensorflow import keras

print("TF  Version",tf.__version__)

In [ ]:
# check and set path before loading modules
print(os.getcwd())
INPUT_DIR = "/tf/notebooks/schnemau/xAI_stroke_3d/"
OUTPUT_DIR = "/tf/notebooks/bule/explainable_AI/"
if os.getcwd() != OUTPUT_DIR:
    os.chdir(OUTPUT_DIR)
    print(os.getcwd())

import functions_metrics as fm
import functions_model_definition as md
import functions_read_data as rdat 
import Utils_maurice as utils
import functions_gradcam as gc
import functions_plot_heatmap as phm

#ontram functions
from k_ontram_functions.ontram import ontram
from k_ontram_functions.ontram_loss import ontram_loss
from k_ontram_functions.ontram_metrics import ontram_acc, ontram_auc
from k_ontram_functions.ontram_predict import predict_ontram, get_parameters

In [ ]:
version = "10Fold_CIB"  # 10Fold_CIBLSX, 10Fold_CIB
layer_connection = "globalAveragePooling" 
last_activation = "linear" 
# Define Model Version
model_version = 1

# should csv be saved?
save_file = False

DATA_OUTPUT_DIR = OUTPUT_DIR + "data/"

In [ ]:
# Setup the paths for the result assembly
DATA_DIR = INPUT_DIR + "data/" 
WEIGHT_DIR = INPUT_DIR + "weights/" + version + "/"
id_tab = pd.read_csv(DATA_DIR + "10Fold_ids_V0.csv", sep=",")
X = np.load(DATA_DIR + "prepocessed_dicom_3d.npy")
all_result_name = "all_tab_results_" + version + "_M" + str(model_version)
which_splits = list(range(0,10)) # 10 Fold
print(id_tab.shape)
print(X.shape)

In [ ]:
(X_train, X_valid, X_test),(X_tab_train, X_tab_valid, X_tab_test), (y_train, y_valid, y_test) = rdat.split_data_tabular(
    id_tab, X, 1)

input_dim = (128, 128, 28, 1)
output_dim = 1
batch_size = 6
C = 2 

mbl = utils.img_model_linear_final(input_dim, output_dim)
if version == "10Fold_CIBLSX":
    mls = utils.mod_linear_shift(X_tab_train.shape[1])
    model_3d = ontram(mbl, mls)
else:
    model_3d = ontram(mbl)             

model_3d.compile(optimizer=keras.optimizers.Adam(learning_rate=5*1e-5),
                                loss=ontram_loss(C, batch_size),
                                metrics=[ontram_acc(C, batch_size)])


In [ ]:
last_activation = "linear"
model_name = ("/tf/notebooks/schnemau/xAI_stroke_3d/weights/" + version + "/3d_cnn_binary_model_split" + 
    ("CIB_LSX" if version == "10Fold_CIBLSX" else "") + str(1) + 
    ("_normalized" if version == "10Fold_CIBLSX" else "_unnormalized") +
    "_avg_layer_paper_model_" + last_activation + "_activation_" + str(1) + "_" + str(1) + ".h5")        

model_3d.load_weights(model_name)        
test_data = tf.data.Dataset.from_tensor_slices((X_test, X_tab_test))
test_labels = tf.data.Dataset.from_tensor_slices((to_categorical(y_test)))
test_loader = tf.data.Dataset.zip((test_data, test_labels))
test_dataset_pred = (test_loader.batch(len(X_test)))
predic =  predict_ontram(model_3d, data = test_dataset_pred)['pdf'][:,0]


In [ ]:
id_num = 10

# ht, im = gc.ontram_grad_cam_3d(X_test[id_num:id_num+1], model_3d, "conv3d_3", "dense_complex_intercept")
ht, im = gc.grad_cam_3d(X_test[id_num:id_num+1], model_3d, "conv3d_3", gcplusplus=True, inv_hm = True)

print("p_id:", id_tab[id_tab.fold1 == "test"].p_id.iloc[id_num])
print("unfavorable:", id_tab[id_tab.fold1 == "test"].unfavorable.iloc[id_num])
print("prediction:", 1-predic[id_num])
phm.plot_heatmap(im, ht,
            version = "overlay",
            mode = "avg",
            hm_colormap="jet")


In [ ]:
phm.plot_heatmap(im, ht,
            version = "original",
            mode = "avg",
            hm_colormap="jet",
            add_orientation=True)

### Check Occlusion

In [ ]:
(res_table, res_images, res_model_names) = gc.get_img_and_models(
        p_ids = [5], results = all_results_tab, pats = pat_ids, imgs = X_in, 
        gen_model_name = generate_model_name,
        num_models = num_models)

In [ ]:
import functions_metrics as fm

def predict_ontram_ensemble(X, model_mode = "weighted",
                            res_tab = res_table, tabular_df = pat_norm_tab, model_names = res_model_names[0], cnn = model_3d):
    
    X = X.reshape(1,128,128,28,1)

    if model_mode == "weighted":
        weights = res_tab.loc[:, res_tab.columns.str.startswith("weight")].to_numpy().squeeze()
        model_names = list(np.array(model_names)[weights > 0])
        weights = weights[weights>0] 
    else:
        weights = np.repeat(1/len(model_names), len(model_names))
    
    out = []
    for model_name in model_names:
        cnn.load_weights(model_name)

        filtered_df = tabular_df[tabular_df['p_id'] == res_tab['p_id'][0]].drop('p_id', axis=1).values
        X_tab_occ = np.tile(filtered_df, (len(X), 1))

        occ_dataset_pred = ((X, X_tab_occ))
        preds = cnn.predict(occ_dataset_pred)
        out.append(preds[:,0]-preds[:,1])
        
    return 1-fm.sigmoid(np.average(np.array(out).squeeze(), weights=weights))

In [ ]:
occluded_img = res_images.copy()

xx = 0
yy = 0
zz = 10

occluded_img[0, xx:xx+occ_size[0], yy:yy+occ_size[1], zz:zz+occ_size[2], 0] = 2

phm.plot_heatmap(occluded_img[0], occluded_img[0],
                version = "original",
                mode = "avg",
                hm_colormap="jet",
                hm_positive=True,
                slice_line=False)

In [ ]:
res_table.y_pred_trafo_avg_w

In [ ]:
predict_ontram_ensemble(occluded_img[0])

In [ ]:
(heatmap, resized_img, max_hm_slice, hm_mean_std, all_heatmaps) =  oc.volume_occlusion(
            volume = res_images, 
            res_tab = res_table, 
            occlusion_size = np.array(occ_size), 
            cnn = model_3d,
            invert_hm="pred_class",
            tabular_df=pat_norm_tab,
            model_mode = "weighted",
            both_directions=False,
            model_names = res_model_names[0],
            normalize = False,
            occlusion_stride = occ_stride)